In [ ]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import datetime
import requests

# Sample corpus for the chatbot to learn from
corpus = [
    'What is your name?',
    'My name is Chatbot. How can I assist you?',
    'How are you?',
    'I am fine. Thank you for asking.',
    'Can you help me?',
    'Sure! I am here to help you. What do you need assistance with?',
    'Goodbye!',
    'Goodbye! Have a great day!',
    'What time is it?',
    'It is currently {time}.',
    'What is the date today?',
    'Today is {date}.',
    # Add more responses for other common questions or greetings
]

# Custom responses for specific queries
custom_responses = {
    'how are you': 'I am just a machine, so I don\'t have feelings, but I\'m here to assist you!',
    'what can you do': 'I can help you with various tasks like answering questions, providing information, and more.',
    'thank you': 'You\'re welcome!'
}

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

# Function to get the response from the chatbot
def get_response(user_input):
    # Check for custom responses first
    for key in custom_responses:
        if key in user_input.lower():
            return custom_responses[key]

    # Check for time
    if 'time' in user_input.lower():
        current_time = datetime.datetime.now().strftime('%H:%M')
        return f"It is currently {current_time}."

    # Check for date
    if 'date' in user_input.lower():
        current_date = datetime.datetime.now().strftime('%Y-%m-%d')
        return f"Today is {current_date}."

    # Check for Wikipedia search
    if 'wikipedia' in user_input.lower():
        query = user_input.replace('wikipedia', '').strip()
        response = requests.get(f"https://en.wikipedia.org/w/api.php?action=query&format=json&list=search&srsearch={query}")
        data = response.json()
        if 'query' in data and 'search' in data['query'] and data['query']['search']:
            search_results = data['query']['search']
            return f"Here is what I found on Wikipedia: https://en.wikipedia.org/wiki/{search_results[0]['title'].replace(' ', '_')}"
        else:
            return "Sorry, I couldn't find information on Wikipedia for that query."

    # Check for joke
    if 'joke' in user_input.lower():
        response = requests.get("https://official-joke-api.appspot.com/jokes/random")
        data = response.json()
        return f"Here's a joke for you: {data['setup']} {data['punchline']}"

    # Check for basic calculations
    if any(op in user_input for op in ['+', '-', '*', '/']):
        try:
            result = eval(user_input)
            return f"The result is {result}."
        except Exception as e:
            return "Sorry, I couldn't perform the calculation."

    # Tokenize and vectorize user input
    user_input = nltk.word_tokenize(user_input.lower())
    user_input = ' '.join([word for word in user_input if word.isalnum()])
    user_input_vectorized = vectorizer.transform([user_input])

    # Calculate cosine similarities between user input and corpus
    cosine_similarities = cosine_similarity(user_input_vectorized, X)

    # Get the index of the most similar sentence
    max_index = np.argmax(cosine_similarities)

    # Return the response
    return corpus[max_index]

# Main function to run the chatbot
def main():
    print("Chatbot: Hello! I'm here to assist you. Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye! Have a great day!")
            break
        response = get_response(user_input)
        print("Chatbot:", response)

if __name__ == "__main__":
    main()



Chatbot: Hello! I'm here to assist you. Type 'exit' to end the conversation.
You: hi
Chatbot: What is your name?
You: my name is kartik
Chatbot: My name is Chatbot. How can I assist you?
You: how are you
Chatbot: I am just a machine, so I don't have feelings, but I'm here to assist you!
You: what can you do
Chatbot: I can help you with various tasks like answering questions, providing information, and more.
You: time
Chatbot: It is currently 18:01.
You: date
Chatbot: Today is 2024-04-29.
You: joke
Chatbot: Here's a joke for you: What does a clock do when it's hungry? It goes back four seconds!
You: wikipedia virat kohli
Chatbot: Here is what I found on Wikipedia: https://en.wikipedia.org/wiki/Virat_Kohli
You: goodbye
Chatbot: Goodbye!
